In [1]:
import itertools
import pathlib

import yaml
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.transforms
from sklearn.metrics import accuracy_score

import lab.metrics
import lab.analytics

sns.set_style("white")
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 8,
    'legend.fontsize': 8,
})
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "sans-serif",
#     "font.sans-serif": ["Helvetica"]})

WITH_GRID = False

In [2]:
INPUTS = [
    f"../../results/ml-eval-mconn/{path}/predict/{classifier}.csv"
    for classifier in ["kfp-0", "dfnet-0", "varcnn-0"]
    for path in ["tamaraw", "undefended", "front"]
]
OUTPUT_FILE = pathlib.Path("/tmp/plot.png")
WITH_LEGEND = True
WITH_SIMULATED = True

if "snakemake" in globals():
    INPUTS = list(snakemake.input)
    OUTPUT_FILE = pathlib.Path(snakemake.output[0])
    WITH_LEGEND = snakemake.params.get("with_legend", False)
    WITH_SIMULATED = snakemake.params.get("with_simulated", True)

In [3]:
def _load_data(inputs):
    data = (pd.concat([
                pd.read_csv(fname, header=0) for fname in inputs
            ], keys=inputs, names=["path"])
            .fillna(0)
            .rename(columns=lambda c: int(c) if c != "y_true" else c)
            .astype({"y_true": int})
            .assign(
                is_simulated=lambda x: x.index.get_level_values("path").str.contains("simulated"),
                defence=lambda x: x.index.get_level_values("path").map(
                    lambda s: next(defence for defence in ["front", "tamaraw", "undefended"] if defence in s)
                ),
                classifier=lambda x: x.index.get_level_values("path").map(
                    lambda s: next(classifier for classifier in ["kfp", "dfnet", "varcnn"] if classifier in s)
                ))
            .set_index(["defence", "classifier", "is_simulated"])
            .rename(index={"front": "FRONT", "tamaraw": "Tamaraw", "undefended": "Undefended",
                           "kfp": "$k$-FP", "dfnet": "DF", "varcnn": "Var-CNN"})
           )
    return data

data = _load_data(INPUTS)
data

y_true             0             1  \
defence classifier is_simulated                                       
Tamaraw $k$-FP     False              0  1.000000e+00  0.000000e+00   
                   False              0  1.000000e+00  0.000000e+00   
                   False              0  1.000000e+00  0.000000e+00   
                   False              0  1.000000e+00  0.000000e+00   
                   False              0  1.000000e+00  0.000000e+00   
...                                 ...           ...           ...   
FRONT   Var-CNN    False             99  1.449788e-10  7.112789e-12   
                   False             99  4.184397e-06  5.538006e-07   
                   False             99  4.292675e-07  2.905551e-07   
                   False             99  4.570857e-07  1.488485e-06   
                   False             99  2.174823e-10  1.972107e-09   

                                            2             3             4  \
defence classifier is_simulated                                             
Tamaraw $k$-FP     False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
...                                       ...           ...           ...   
FRONT   Var-CNN    False         2.015747e-15  1.762092e-12  1.522975e-09   
                   False         7.704123e-08  4.434974e-06  5.755274e-07   
                   False         5.897025e-10  2.606692e-07  1.317687e-07   
                   False         3.778928e-09  1.307442e-06  2.673777e-06   
                   False         1.003525e-13  2.147420e-11  2.757162e-08   

                                            5             6             7  \
defence classifier is_simulated                                             
Tamaraw $k$-FP     False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
                   False         0.000000e+00  0.000000e+00  0.000000e+00   
...                                       ...           ...           ...   
FRONT   Var-CNN    False         2.173863e-09  2.596373e-17  1.526077e-08   
                   False         4.778302e-04  3.051396e-10  1.209225e-04   
                   False         3.341492e-05  2.672154e-11  3.227483e-06   
                   False         7.911517e-06  3.941725e-10  1.430952e-04   
                   False         2.741972e-08  2.670440e-15  6.093600e-09   

                                            8  ...            90  \
defence classifier is_simulated                ...                 
Tamaraw $k$-FP     False         0.000000e+00  ...  0.000000e+00   
                   False         0.000000e+00  ...  0.000000e+00   
                   False         0.000000e+00  ...  0.000000e+00   
                   False         0.000000e+00  ...  0.000000e+00   
                   False         0.000000e+00  ...  0.000000e+00   
...                                       ...  ...           ...   
FRONT   Var-CNN    False         3.679536e-09  ...  9.411934e-09   
                   False         3.595397e-05  ...  2.680891e-05   
                   False         9.541577e-06  ...  2.475094e-06   
                   False         3.136531e-05  ...  1.814584e-05   
                   False         3.269045e-09  ...  1.569175e-08   

                                           91            92            93  \
defence classifier is_simulated                                             
Tamaraw $k$-FP     False         0.000000e+00  0.000000e+00  0.000

In [5]:
(data.groupby(["defence", "classifier", "is_simulated"])
 .apply(lambda df: accuracy_score(df["y_true"], df.iloc[:, 1:].idxmax(axis=1)))
 .unstack("classifier"))

# data["y_pred"] = data.iloc[:, 1:].idxmax(axis=1)
# data

,classifier,$k$-FP,DF,Var-CNN
defence,is_simulated,,,
FRONT,False,0.7000,0.8790,0.9180
Tamaraw,False,0.8910,0.8525,0.8705
Undefended,False,0.9895,0.9880,0.9925
